# Web scraping from trabajopolis.bo

<h3> Step1: Import libraries </h3>

In [4]:
import requests
from bs4 import BeautifulSoup
import pymysql

<h3>Step 2: Request a job and check the response status code to see if everything went as planned</h3>
<li>status code 200: the request response cycle was successful
<li>any other status code: it didn't work (e.g., 404 = page not found)

In [5]:
keywords = input("Please enter the job you want to search: ")
#url = "http://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D=" + keywords
url = "http://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D="+ keywords + "&page=1&listings_per_page=100"
response = requests.get(url)
if response.status_code == 200:
    print("The request response cycle was successful")
else:
    print("It didn't work")

Please enter the job you want to search: Programador
The request response cycle was successful


In [6]:
results_page = BeautifulSoup(response.content,'lxml')
#print(results_page.prettify())

<h3>Step 3: Extracting job link related to a request</h3>
<li>I look for all 'td' tags and 'anuncio-estandar-content'</li>
<li>I create a list with the job name and link

In [7]:
results_page = BeautifulSoup(response.content,'lxml')
trabajo_list = list()
trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
for anuncio in trabajos:
    trabajo_link = anuncio.find('a').get('href')
    trabajo_name = anuncio.find('b').get_text()
    trabajo_list.append((trabajo_name,trabajo_link))

In [8]:
trabajo_list

[('Programador(a) de Sistema Contable',
  'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/691721/Programador(a)-de-Sistema-Contable.html?searchId=1545846372.21&page=1'),
 ('Programador(a)',
  'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/686533/Programador(a).html?searchId=1545846372.21&page=1'),
 ('Programador(a) Sénior - Santa Cruz',
  'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/692283/Programador(a)-Sénior---Santa-Cruz.html?searchId=1545846372.21&page=1')]

# Let's write a function that given a job request, returns a list containing the key factors
Dos funciones:
<li>get_job_info
<li>trabajopolis

In [9]:
def get_job_info(trabajo_link):
    """El input es un enlace que dirige a una única oferta de trabajo. Devuelve un tuple con la información más relevante: d
    escripción del anuncio y del trabajo""""
    anuncio_list = []
    description_list = []
    import requests
    from bs4 import BeautifulSoup
    try:
        response = requests.get(trabajo_link)
        if not response.status_code == 200:
            return trabajo_dict
        result_page = BeautifulSoup(response.content,'lxml')
        for info in result_page.find_all('div', class_='smallListingInfo'): 
            #esto corresponde a la descripción del anuncio: ID Empleo, Categoria, Ubicación, etc
            anuncio_list.append(info.get_text())
        for trabajo in result_page.find_all('p',class_='MsoNormal'):
            #esto corresponde a la descripción del anuncio: Requisitos de formación, experiencia, etc
            description_list.append(trabajo.get_text())
        return (anuncio_list, description_list)
    except:
        return None

In [29]:
get_job_info('https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/691721/Programador(a)-de-Sistema-Contable.html')

(['ID Empleo: 691721',
  'Ubicación: Santa Cruz de la Sierra, Bolivia',
  'Categoría: Informática ',
  'Sueldo: Negociable ',
  'Contrato: Tiempo Completo, Medio Tiempo',
  '\nPublicado: 12 Diciembre 2018\n',
  '\nVencimiento: 11 Enero 2019\n\n'],
 [])

In [10]:
def trabajopolis(job_description):
    """ A partir de una (o más) palabras claves  """
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    url = "http://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D="+ job_description + "&page=1&listings_per_page=100"
    
    response = requests.get(url)
    if response.status_code == 200:
        print("The request response cycle was successful")
        results_page = BeautifulSoup(response.content,'lxml')
        #trabajo_list = list()
        trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
        for anuncio in trabajos:
            trabajo_link = anuncio.find('a').get('href')
            trabajo_name = anuncio.find('b').get_text()
            print(trabajo_name)
         #   trabajo_list.append((trabajo_name,trabajo_link))
        #for trabajo in trabajo_list:
            (anuncio_list, description_list) = get_job_info(trabajo_link)
            trabajos_list.append((anuncio_list,trabajo_name,trabajo_link, description_list))
        #I need to create a list that save all the extracted data
    else:
        print("The request response cycle work didn't work")
    return trabajos_list

# Output is below

In [22]:
Results = []
palabras_clave = ["secretaria", "recepcionista"] 
for busqueda in palabras_clave:
    Results.append(trabajopolis(busqueda))

The request response cycle was successful
Secretaria Recepcionista
Secretaria
Secretaria Administrativa
Secretaria - Oruro
Secretaria a Tiempo Completo - Santa Cruz de la Sierra
Secretaria
The request response cycle was successful
Secretaria Recepcionista
Recepcionista - Santa Cruz
Recepcionista de Hotel
Recepcionista (Temporal) - Santa Cruz
Recepcionista - Santa Cruz
Recepcionista (Temporal) - La Paz
Recepcionista - Santa Cruz de la Sierra
Recepcionista - Turno Mañana


In [11]:
Results = trabajopolis("ingeniero")
#Results = []
#palabras_clave = ["secretaria", "recepcionista"] 
#for busqueda in palabras_clave:
#    Results.append(trabajopolis(busqueda))

The request response cycle was successful
Ingeniero(a) en Sistema - Santa Cruz
Supervisor(a) de Obras Civiles
Supervisor(a) de Mantenimiento
Representante Comercial
Soporte Técnico - Santa Cruz
Ingeniero(a) Civil
Ingeniero(a) de Ventas - Santa Cruz
Ingeniero(a) Comercial
Ejecutivo(a) de Ventas - Área Hidráulica
Ejecutivo(a) de Ventas - Santa Cruz
Ingeniero(a) Eléctrico Jr. para el Área de Ventas y Soporte
Ingeniero(a) de Sistemas
Ingeniero(a) Químico(a)
Ingeniero(a) en Sistemas
Ingeniero(a) de Preventa TI - La Paz
Ingeniero(a) para Elaboración de Proyectos - Product Manager
Asistente de Servicio Técnico - Santa Cruz
Ingeniero(a) de Productos (Ventas)
Ingeniero(a) Químico(a)
Analista de Sistemas
Vendedor(a) Oil - Free Air Compressor Technique Division - La Paz
Ingeniero(a) de Calidad de Experiencia al Cliente
Gerente de Planta - Cochabamba
Ingeniero(a) de Servicios
Especialista de Sistemas
Analista de Inteligencia de Clientes y Datos - Santa Cruz
Ingeniero(a) Civil para Supervisión y El

In [28]:
Results[0]

[(['ID Empleo: 696418',
   'Ubicación: Santa Cruz de la Sierra, Bolivia',
   'Categoría: Administración y Oficina,Atención al Cliente ',
   'Sueldo: No Declarado ',
   'Contrato: Tiempo Completo',
   '\nPublicado: 22 Diciembre 2018\n',
   '\nVencimiento: 21 Enero 2019\n\n'],
  'Secretaria Recepcionista',
  'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/696418/Secretaria-Recepcionista.html?searchId=1545848368.01&page=1',
  []),
 (['ID Empleo: 695082',
   'Ubicación: Santa Cruz de la Sierra, Bolivia',
   'Categoría: Administración y Oficina ',
   'Sueldo: No Declarado ',
   'Contrato: Tiempo Completo',
   '\nPublicado: 18 Diciembre 2018\n',
   '\nVencimiento: 17 Enero 2019\n\n'],
  'Secretaria',
  'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/695082/Secretaria.html?searchId=1545848368.01&page=1',
  []),
 (['ID Empleo: 694770',
   'Ubicación: Santa Cruz de la Sierra, Bolivia',
   'Categoría: Administración y Oficina,Atención al Cliente ',
   'Sueld

In [14]:
len(Results)
Results[1][0]

['ID Empleo: 696408',
 'Ubicación: Potosí, Bolivia',
 'Categoría: Construcción,Ingeniería ',
 'Sueldo: No Declarado ',
 'Contrato: Tiempo Completo',
 '\nPublicado: 22 Diciembre 2018\n',
 '\nVencimiento: 21 Enero 2019\n\n']

# Remove unnecessary space
(Necesitaré mejorar esto, es provisional. 
Puesto que Results[0][0][6] son strings puedo eliminar las letras innecesarias antes de pasarlo a CSV). Encontré un caso en el que falta uno de los datos, ver como proceder.

In [17]:
Results
for i in range(len(Results)):
    try:
        Results[i][0][0] =  Results[i][0][0][11:] #Elimina la parte de Empleo ID:
        #Results[i][0][1] = Results[i][0][1][11:] #Elimina la palabra Ubicación
        #Results[i][0][2] =  Results[i][0][2][11:] #Elimina la palabra categoria
        #Results[i][0][3] = Results[i][0][3][8:]
        #Results[i][0][4] = Results[i][0][4][10:]
        Results[i][0][5] =  Results[i][0][5][1:-1]
        Results[i][0][6] = Results[i][0][6][1:-2]
    except:
        continue
    print(i)
Results

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
25
26
27
28
29
30


[(['696421',
   'Ubicación: Santa Cruz de la Sierra, Bolivia',
   'Categoría: Informática,Ingeniería ',
   'Sueldo: No Declarado ',
   'Contrato: Tiempo Completo',
   'ublicado: 24 Diciembre 201',
   'encimiento: 23 Enero 20'],
  'Ingeniero(a) en Sistema - Santa Cruz',
  'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/696421/Ingeniero(a)-en-Sistema---Santa-Cruz.html?searchId=1545846400.1&page=1',
  []),
 (['696408',
   'Ubicación: Potosí, Bolivia',
   'Categoría: Construcción,Ingeniería ',
   'Sueldo: No Declarado ',
   'Contrato: Tiempo Completo',
   'ublicado: 22 Diciembre 201',
   'encimiento: 21 Enero 20'],
  'Supervisor(a) de Obras Civiles',
  'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/696408/Supervisor(a)-de-Obras-Civiles.html?searchId=1545846400.1&page=1',
  []),
 (['696293',
   'Ubicación: Santa Cruz de la Sierra, Bolivia',
   'Categoría: Otros ',
   'Sueldo: No Declarado ',
   'Contrato: Tiempo Completo',
   'ublicado: 21 Diciembre 20

# Save in csv file
For now it doesn't save the "description_list" since I clean data before saving it
In the future I will use SQL

In [18]:
import csv
outfile = open("/Users/Roberto/Documents/Online Learning/Python programming/Trabajopolis scraper/trabajo.csv", 'w',newline='')
writer = csv.writer(outfile)
try:
    for i in range(len(Results)):
        csvRow = []
        for cell in Results[i][0]:
            #print(cell)
            csvRow.append(cell)
        csvRow.append(Results[i][1])
        csvRow.append(Results[i][2])
        print(csvRow)
        writer.writerow(csvRow)
finally:
    outfile.close() 

['696421', 'Ubicación: Santa Cruz de la Sierra, Bolivia', 'Categoría: Informática,Ingeniería ', 'Sueldo: No Declarado ', 'Contrato: Tiempo Completo', 'ublicado: 24 Diciembre 201', 'encimiento: 23 Enero 20', 'Ingeniero(a) en Sistema - Santa Cruz', 'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/696421/Ingeniero(a)-en-Sistema---Santa-Cruz.html?searchId=1545846400.1&page=1']
['696408', 'Ubicación: Potosí, Bolivia', 'Categoría: Construcción,Ingeniería ', 'Sueldo: No Declarado ', 'Contrato: Tiempo Completo', 'ublicado: 22 Diciembre 201', 'encimiento: 21 Enero 20', 'Supervisor(a) de Obras Civiles', 'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/696408/Supervisor(a)-de-Obras-Civiles.html?searchId=1545846400.1&page=1']
['696293', 'Ubicación: Santa Cruz de la Sierra, Bolivia', 'Categoría: Otros ', 'Sueldo: No Declarado ', 'Contrato: Tiempo Completo', 'ublicado: 21 Diciembre 201', 'encimiento: 20 Enero 20', 'Supervisor(a) de Mantenimiento', 'https://www.tra